In [1]:
#Homework 4
#Ethan Witkowski

In [1]:
import numpy as np
import random as rnd
from statistics import mean
from psychopy import visual, core, event

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# 1

In [2]:
#Randomizes sequence of conditions across blocks of an experiement
#No consecutive repeats of conditions

def exp_randomization(nblocks,nconds):
    block_list = []
    
    #Initialize condition
    init_condition = np.random.choice(range(1,nconds+1), size=(1,nconds), replace=False, p=None)
    block_list.append(init_condition)

    for block in range(nblocks-1):

        while True:

            #Random condition sequence for current block
            condition_list = np.random.choice(range(1,nconds+1), size=(1,nconds), replace=False, p=None)

            prior_block = block_list[-1]
            
            #If consective conditions, choose new random condition
            if condition_list[0][0] == prior_block[-1][-1]:
                continue    

            #Append random condition to list of blocks
            else:
                block_list.append(condition_list)
                break

    print(block_list)

In [3]:
nblocks = input("Please input the number of blocks: ")
nconds = input("Please input the number of conditions: ")

Please input the number of blocks: 3
Please input the number of conditions: 3


In [6]:
nblocks = int(nblocks)
nconds = int(nconds)

exp_conditions = exp_randomization(nblocks,nconds)

[array([[2, 3, 1]]), array([[3, 1, 2]]), array([[3, 2, 1]])]


# 2

In [7]:
#Read-in file
f = open('midterm_catfr_data.txt', 'r')
full_data = f.readlines()
f.close()

In [8]:
#Remove spacing elements, transfer to list
for i, observation in enumerate(full_data):
    full_data[i] = observation.strip().split(',')
    
print(full_data[i])

['R1449T', 'catFR1', '1', '12', '12', 'DOVE', 'Birds', '1']


In [9]:
#list to hold variable values
data = []

for i in range(len(full_data[0])):              
    data.append([])                   #Create new list for each variable

    for sub_list in full_data:
        data[i].append(sub_list[i])   #Append ith element in sub_list to correct variable

#Declare variable names
subjects, experiment, sess, list_number, serial_position, word_pres, category, recall = data

#List of unique categories
unique_category = list(set(category))

#List of unique subjects
unique_subjects = list(set(subjects))

In [10]:
#Trial level data np arrays

np_categories = np.array(category)
np_subjects = np.array(subjects)
np_recall = np.array(recall)

#Unique categories/subjects
np_u_categories = np.unique(np_categories)
np_u_subjects = np.unique(np_subjects)

In [11]:
#Pre-allocate 2d array to store means of each subjects/category

np_subjects_mean_recall_by_cat = np.empty([186,26])

In [12]:
#Obtain each subject's performance recall for each category

for i,u_subjects in enumerate(np_u_subjects):
    
    for j,u_categories in enumerate(np_u_categories):
        
        #Inner np.logical_and: 70,000 long, True if subj/cat match loops
        subj_and_cat_match = np.logical_and(np_subjects==u_subjects,np_categories==u_categories)
        
        #indexing np_recall with True, gives us 0/1s for each subject for each category
        recall_index = np_recall[subj_and_cat_match]
        
        #Convert to float
        recall_index = recall_index.astype(np.float)
        
        #Given recall values, taking the mean gives us their performance recall
        #Place in numpy array corresponding to subject/category
        if len(recall_index) > 0:
            np_subjects_mean_recall_by_cat[i,j] = np.mean(recall_index)
            
        else:
            continue

In [13]:
#Transpose for arrays to represent categories, elements in arrays the mean recall for each subject
np_t_subjects_mean_recall_by_cat = np.transpose(np_subjects_mean_recall_by_cat)
np.shape(np_t_subjects_mean_recall_by_cat)

(26, 186)

In [14]:
#Pre-allocate matrix of correlation coefficients for each subject for each category

corr_cat_matrix = np.empty([26,26])
corr_cat_matrix[:] = np.nan

In [15]:
#Place category recall correlation coefficients in matrix

for m,cat in enumerate(np_t_subjects_mean_recall_by_cat):

    for n,cat_2 in enumerate(np_t_subjects_mean_recall_by_cat):
        corr_cat_matrix[m][n] = np.corrcoef(cat,cat_2)[1,0]

print(len(corr_cat_matrix))
print(len(corr_cat_matrix[0]))
print(corr_cat_matrix)

26
26
[[ 1.          0.33620532  0.47344002  0.57628744  0.44385186  0.40555807
   0.34773137  0.3012153  -0.02958102  0.39140204  0.37793233  0.44829428
   0.44017189  0.32723223  0.49451358  0.41945275  0.45664382  0.25059239
   0.41820855  0.47016521  0.43204532  0.35789562  0.32501954  0.30467306
   0.33924985  0.42425596]
 [ 0.33620532  1.          0.31899512  0.51651602  0.35514625  0.36644797
   0.42716067  0.27507438 -0.02172599  0.48325054  0.36920256  0.42352439
   0.37505843  0.38715487  0.44214568  0.40236885  0.4638761   0.36101879
   0.49222841  0.35074932  0.47487193  0.39731933  0.22278141  0.43071002
   0.42488471  0.3419053 ]
 [ 0.47344002  0.31899512  1.          0.40527716  0.49067997  0.34963534
   0.39285716  0.177379    0.03746076  0.4406243   0.42250268  0.43289163
   0.42228666  0.31725066  0.34008002  0.34785715  0.4586931   0.35041043
   0.36173086  0.42301312  0.3144575   0.35951562  0.23255939  0.35043107
   0.34507766  0.45759506]
 [ 0.57628744  0.51651602

In [16]:
#Find maximum correlation coefficient, pair of categories

corr_max = 0

for i,corr_array in enumerate(corr_cat_matrix):
    
    for j,corr_value in enumerate(corr_array):
        if (corr_value > corr_max) & (corr_value < 0.99999):
            corr_max = corr_value
            max_i_index = (i)
            max_j_index = (j)
        else:
            continue

print("The maximum correlation coefficient is " + str(corr_max))

print("The pair of categories with the highest correlation in recall performance is: ") 
print(str(np_u_categories[max_i_index]) + " and " + str(np_u_categories[max_j_index]))

The maximum correlation coefficient is 0.6163920742941553
The pair of categories with the highest correlation in recall performance is: 
Flowers and KitchenTools


In [17]:
#Find minimum correlation coefficient, pair of categories

corr_min = 2

for i,corr_array in enumerate(corr_cat_matrix):
    
    for j,corr_value in enumerate(corr_array):
        if (abs(corr_value) < abs(corr_min)):
            corr_min = corr_value
            min_i_index = (i)
            min_j_index = (j)
        else:
            continue

print("The minimum correlation coefficient is " + str(corr_min))

print("The pair of categories with the lowest correlation in recall performance is: ") 
print(str(np_u_categories[min_i_index]) + " and " + str(np_u_categories[min_j_index]))

The minimum correlation coefficient is 0.010695817917302466
The pair of categories with the lowest correlation in recall performance is: 
Fabric and Toys


# 3

In [2]:
#Read-in word list
f = open('wordpool.txt')
words = f.read().splitlines()
f.close()

#Set number of targets
ntargets = input("Please input the number of targets: ")
ntargets = int(ntargets)

#Set number of lures
nlures = input("Please input the number of lures: ")
nlures = int(nlures)

Please input the number of targets: 5
Please input the number of lures: 5


In [19]:
#Randomly select targets and lures

#Randomly shuffle words, choose targets, delete chosen targets
rnd.shuffle(words)
targets = words[:ntargets]
del words[:ntargets]
print(targets)

#Randomly shuffle remaining words, choose lures
rnd.shuffle(words)
lures = words[:nlures]
del words[:nlures]
print(lures)

#Reset list of words
f = open('wordpool.txt')
words = f.read().splitlines()
f.close()

['CARRY', 'THEATER', 'CONTAIN', 'LONELY', 'OPEN']
['EMBRACE', 'COLONEL', 'IDLE', 'CORRECT', 'RELEASE']


In [20]:
#List of randomly selected targets/lures presented during test phase

#Concatenate lists of targets and lures
test_words = targets + lures
print(test_words)

#Randomly shuffle words
rnd.shuffle(test_words)
print(test_words)

#Pre-allocate numpy array for responses
responses = np.empty([1,len(test_words)], dtype = "object")

#Pre-allocate list of results
results = np.empty([1,len(test_words)], dtype = "object")

['CARRY', 'THEATER', 'CONTAIN', 'LONELY', 'OPEN', 'EMBRACE', 'COLONEL', 'IDLE', 'CORRECT', 'RELEASE']
['IDLE', 'CARRY', 'OPEN', 'RELEASE', 'COLONEL', 'EMBRACE', 'LONELY', 'CORRECT', 'CONTAIN', 'THEATER']


In [23]:
#Experiment
#Encoding phase

#Create window
win = visual.Window([400,400])

#Show target words to participant with 1-second delay
for target in targets:
    stim = visual.TextStim(win,text=target.rstrip('\n'))
    stim.draw()
    win.flip()
    core.wait(1)
    
#Intermediate phase

#Present window asking participant to input 'i' to move to test phase
stim = visual.TextStim(win,text='In the test phase, pressing "j" indicates a word shown in the encoding phase. Pressing "k" indicates a word not shown in the encoding phase. Please press "i" to show your understanding.')
stim.draw()
win.flip()
pressedkey = event.waitKeys(keyList = ["i"])
if pressedkey[0] == 'i':
    win.flip()
    core.wait(1)
    
#Test Phase

for i,test_word in enumerate(test_words):
    stim = visual.TextStim(win,text=test_word.rstrip('\n'))
    stim.draw()
    win.flip()
    pressedkey = event.waitKeys(keyList = ["j", "k"])
    pressedkey = "".join(pressedkey)
    
    responses[0][i] = pressedkey
    
    
    if (responses[0][i] == "j") & (test_words[i] in targets):
        results[0][i] = str("hit")
        
    elif (responses[0][i] == "j") & (test_words[i] in lures):
        results[0][i] = str("false alarm")
     
    elif (responses[0][i] == "k") & (test_words[i] in targets):
        results[0][i] = str("miss")
    
    elif (responses[0][i] == "k") & (test_words[i] in lures):
        results[0][i] = str("correct rejection")

win.close()
print(responses)
print(results)


hit_counter = 0
for result in results[0]:
    if result == "hit":
        hit_counter += 1
    
hit_rate = hit_counter/ntargets
print("The hit rate is " + str(hit_rate))

false_alarm_counter = 0
for result_2 in results[0]:
    if result_2 == "false alarm":
        false_alarm_counter += 1

false_alarm_rate = false_alarm_counter/nlures
print("The false alarm rate is " + str(false_alarm_rate))

194.4218 	WARNING 	Monitor specification not found. Creating a temporary one...
[['k' 'k' 'j' 'k' 'k' 'k' 'j' 'k' 'j' 'j']]
[['correct rejection' 'miss' 'hit' 'correct rejection'
  'correct rejection' 'correct rejection' 'hit' 'correct rejection' 'hit'
  'hit']]
The hit rate is 0.8
The false alarm rate is 0.0
